# Introduction: Business Problem 

I live in Taiwan. There are lots of convenience store in Taiwan, but more and more convenience store will open. Because population density in Taiwan is very high, and Taiwanese is very lazy, one of reason Taiwanese choose where to live is there any convenience store. Therefore more and more convenience stores is required.

In this project, I will try to find better location for convenience store. This report will be targeted to stakeholders interested in opening an convenience store in Yilan, Taiwan.

The location will close to location where population density is high, so that it will keep revenue. And it must be less convenience store already in that location.

# Data 

According above description, the project is similar to template project, and I will keep the idea of clustering the city by area and then plot heatmap to find better area. And I will change data to Yilain, Taiwan.

Thre are two factors that infuleence: 
* number of existing convenience stores in the neighborhood 
* number of and distance to convenience stores in the neighborhood


I will use the following API:
* Foursquare API: to find convenience stores
* Google API: reverse geolocalisation

In [1]:
from IPython.display import Image
import pickle
import json
import requests
import folium
import pandas as pd

# !pip install shapely
import shapely.geometry

# !pip install pyproj
import pyproj

import math

import warnings
warnings.simplefilter("ignore")

In [2]:
GOOGLE_API_KEY = '#######################'

## Get coordinate of yilan

In [3]:
def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Yilan, Yilan, Taiwan'
Yialn_center = get_coordinates(GOOGLE_API_KEY, address)
print('Coordinate of {}: {}'.format(address, Yialn_center))

Coordinate of Yilan, Yilan, Taiwan: [24.7591148, 121.7537404]


### Transpose coordinate of Yilan

In [4]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Yialn center longitude={}, latitude={}'.format(Yialn_center[1], Yialn_center[0]))
x, y = lonlat_to_xy(Yialn_center[1], Yialn_center[0])
print('Yialn center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Yialn center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Yialn center longitude=121.7537404, latitude=24.7591148
Yialn center UTM X=8982458.66410907, Y=13538226.084463917
Yialn center longitude=121.75374040006089, latitude=24.75911479994217


### Visualize city center location and candidate neighborhood centers

In [5]:
Yialn_center_x, Yialn_center_y = lonlat_to_xy(Yialn_center[1], Yialn_center[0]) # City center in Cartesian coordinates
nb_k = 10
radius = 100
k = 2#math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = Yialn_center_x - radius*10
x_step = radius*2 *k
y_min = Yialn_center_y - radius*2 - (int(nb_k/k)*k*radius*2 - radius*10)/2
y_step = radius*2 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(nb_k/k)):
    y = y_min + i * y_step
    x_offset = radius if i%2==0 else 0
    for j in range(0, nb_k):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(Yialn_center_x, Yialn_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)
            
map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.Marker(Yialn_center, popup='Prefecture').add_to(map_Yialn)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=radius, color='blue', fill=False).add_to(map_Yialn)
map_Yialn

In [6]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(GOOGLE_API_KEY, Yialn_center[0], Yialn_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(Yialn_center[0], Yialn_center[1], addr))

Reverse geocoding check
-----------------------
Address of [24.7591148, 121.7537404] is: No. 41, Lixin Road, Yilan City, Yilan County, Taiwan 260


### use Google Maps API to get approximate addresses of those locations.

In [7]:
addresses = []
compteur = 0

df_locations = pd.DataFrame()
loaded = False
try:
    with open('locations.pkl', 'rb') as f:
        df_locations = pickle.load(f)
    print('Location data loaded from pickle.')
    loaded = True
except:
    pass


if not loaded:
    print('Obtaining location addresses: ', end='')
    for lat, lon in zip(latitudes, longitudes):
        compteur = compteur + 1
        address = get_address(GOOGLE_API_KEY, lat, lon)
        if address is None:
            address = 'NO ADDRESS'
        address = address.replace(', France', '') # We don't need country part of address
        addresses.append(address)
        if compteur > 500:
            print("Urgency exit")
            break
    #     print(compteur)
        print(' .', end='')
    print(' done.')

Location data loaded from pickle.


In [8]:
if not loaded:
    df_locations = pd.DataFrame({'Address': addresses,
                                 'Latitude': latitudes,
                                 'Longitude': longitudes,
                                 'X': xs,
                                 'Y': ys,
                                 'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"No. 21, Huanhe Road, Yilan City, Yilan County,...",24.764210,121.753484,8.981559e+06,1.353753e+07,1140.175425
1,"260, Taiwan, Yilan County, Yilan City, 碧霞街146號",24.762748,121.752353,8.981959e+06,1.353753e+07,860.232527
2,"260, Taiwan, Yilan County, Yilan City, 舊城西路125...",24.761287,121.751223,8.982359e+06,1.353753e+07,707.106781
3,"No. 109, Jiucheng West Road, Yilan City, Yilan...",24.759826,121.750092,8.982759e+06,1.353753e+07,761.577311
4,"No. 142, Wenchang Road, Yilan City, Yilan Coun...",24.758364,121.748961,8.983159e+06,1.353753e+07,989.949494
5,"No. 178, Section 2, Minquan Road, Yilan City, ...",24.756903,121.747831,8.983559e+06,1.353753e+07,1303.840481
6,"No. 80, Sanqing Road, Yilan City, Yilan County...",24.755442,121.746700,8.983959e+06,1.353753e+07,1655.294536
7,"No. 73-1, Taishan Road, Yilan City, Yilan Coun...",24.753981,121.745570,8.984359e+06,1.353753e+07,2024.845673
8,"260, Taiwan, Yilan County, Yilan City, 民權新路226-1號",24.752520,121.744440,8.984759e+06,1.353753e+07,2404.163056
9,"No. 8, Yizhong Road, Yilan City, Yilan County,...",24.751059,121.743310,8.985159e+06,1.353753e+07,2789.265136


In [9]:
df_locations.to_pickle('./locations.pkl')    

# Methodology

### use Foursquare API to get info on convenience store in each neighborhood.

In [10]:
foursquare_client_id = 'ZXXJT2W3JU5JMJBT2FEDEZ3NBBHNDBTZ001ACWK1KWTTIJEF'
foursquare_client_secret = 'DMC2HQFZSW3MM52J2XPDVC1F4WHC3QFZPJMPDH1GUCGK4CRD'

In [11]:
# Category IDs corresponding to Italian stores were taken from Foursquare web site 
# (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06378d81259' # 'Root' category for all food-related venues

# We will add some convenience categories, and also take away food, and healthy food. These category are the one that
# may be competitor
convenience_store_categories = ['4d954b0ea243a5684a65b473','52f2ab2ebcbc57f1066b8b46']

def is_store(categories, specific_filter=None):
    store_words = ['convenience', 'department', 'supermarket']
    store = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in store_words:
            if r in category_name:
#                 print('category_name', category_name)
                store = True
        if 'fast food' in category_name:
            store = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            store = True
#             print(category_name, category_id)
    return store, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [12]:
# Let's now go over our neighborhood locations and get nearby stores; we'll also maintain 
# a dictionary of all found stores and all found italian stores

def get_stores(lats, lons):
    from tqdm.autonotebook import tqdm
    tqdm.pandas()
    stores = {}
    convenience_stores = {}
    location_stores = []

    print('Obtaining venues around candidate locations:', end='')
    pbar = tqdm(total=len(lats))
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any store (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, 
                                          foursquare_client_secret, radius=350, limit=100)
#         with open('1. venues.txt', 'w') as outfile:
#             json.dump(venues, outfile)
        area_stores = []
#         print(venues)
        
        for venue in venues:            
#             with open('2. venue.txt', 'w') as outfile:
#                 json.dump(venue, outfile)
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_convenience = is_store(venue_categories, specific_filter=convenience_store_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                store = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, 
                              venue_distance, is_convenience, x, y)
#                 print("\n" + str(store))
                if venue_distance<=100:
                    area_stores.append(store)
                stores[venue_id] = store
                if is_convenience:
                    convenience_stores[venue_id] = store
        pbar.update(1)        
        
        location_stores.append(area_stores)
#         print(' .', end='')
    pbar.close()
#     print(' done.')
    return stores, convenience_stores, location_stores

# Try to load from local file system in case we did this before
stores = {}
convenience_stores = {}
location_stores = []
loaded = False
try:
    with open('stores_350.pkl', 'rb') as f:
        stores = pickle.load(f)
    with open('convenience_stores_350.pkl', 'rb') as f:
        convenience_stores = pickle.load(f)
    with open('location_stores_350.pkl', 'rb') as f:
        location_stores = pickle.load(f)
    print('store data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    stores, convenience_stores, location_stores = get_stores(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('stores_350.pkl', 'wb') as f:
        pickle.dump(stores, f)
    with open('convenience_stores_350.pkl', 'wb') as f:
        pickle.dump(convenience_stores, f)
    with open('location_stores_350.pkl', 'wb') as f:
        pickle.dump(location_stores, f)

Obtaining venues around candidate locations:

  0%|          | 0/50 [00:00<?, ?it/s]

In [13]:
import numpy as np

print('Total number of stores:', len(stores))
print('Total number of convenience stores:', len(convenience_stores))
print('Percentage of convenience stores: {:.2f}%'.format(len(convenience_stores) / len(stores) * 100))
print('Average number of stores in neighborhood:', np.array([len(r) for r in location_stores]).mean())

Total number of stores: 27
Total number of convenience stores: 22
Percentage of convenience stores: 81.48%
Average number of stores in neighborhood: 0.24


### Visualize convenience stores, see all potential competitors

In [14]:
map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.Marker(Yialn_center, popup='Prefecture').add_to(map_Yialn)
for res in stores.values():
    lat = res[2]; lon = res[3]
    is_convenience = res[6]
    color = 'red' if is_convenience else 'blue'
    label = '{}'.format(res[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_Yialn)
map_Yialn

### visualize universities

In [19]:
# 'Root' category for all universities venues
univ_category = ['4d4b7105d754a06372d81259']

def get_meta_venues(lats, lons, meta_category):
    from tqdm.autonotebook import tqdm
    tqdm.pandas()
    meta_venues = {}
    neighborhoods_venues = []

#     print('Obtaining venues around candidate locations:', end='')
    pbar = tqdm(total=len(lats), desc = 'Obtaining venues', unit= ' coord')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any store (we're using dictionaries to remove any duplicates resulting from area overlaps)
        area_meta_venues = []
        for i, category in enumerate(meta_category):
            venues = get_venues_near_location(lat, lon, category, foursquare_client_id, 
                                              foursquare_client_secret, radius=350, limit=100)
            for venue in venues:
                venue_id = venue[0]
                venue_name = venue[1]
                venue_categories = venue[2]
                venue_latlon = venue[3]
                venue_address = venue[4]
                venue_distance = venue[5]

                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                store = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], 
                              venue_address, venue_distance, is_convenience, x, y)
                if venue_distance<=100:
                    area_meta_venues.append(store)
                meta_venues[venue_id] = store

            neighborhoods_venues.append(area_meta_venues)
#         print(' .', end='')
        pbar.update(1)
    pbar.close()
#     print(' done.')
    return meta_venues, neighborhoods_venues



# plantage = plantage # Just to make sure we won't go after this point

# Try to load from local file system in case we did this before
meta_univ = {}
neighborhoods_univ = []
loaded = False
try:
    with open('meta_univ_350.pkl', 'rb') as f:
        meta_univ = pickle.load(f)
    with open('neighborhoods_univ_350.pkl', 'rb') as f:
        neighborhoods_univ = pickle.load(f)
    print('Universities data loaded.')
    loaded = True
except:
    pass

if not loaded:
    meta_univ, neighborhoods_univ = get_meta_venues(latitudes, longitudes, univ_category)
    
    # Let's persists this in local file system
    with open('meta_univ_350.pkl', 'wb') as f:
        pickle.dump(meta_univ, f)
    with open('neighborhoods_univ_350.pkl', 'wb') as f:
        pickle.dump(neighborhoods_univ, f)
        
print('Total number of universities:', len(meta_univ))
print('Average number of universities in neighborhood:', np.array([len(r) for r in neighborhoods_univ]).mean())

map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.Marker(Yialn_center, popup='Prefecture').add_to(map_Yialn)
for res in meta_univ.values():
    lat = res[2]; lon = res[3]
    is_univ = res[6]
    color = 'blue' 
    label = '{}'.format(res[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_Yialn)
map_Yialn

Obtaining venues:   0%|          | 0/50 [00:00<?, ? coord/s]

Total number of universities: 12
Average number of universities in neighborhood: 0.1


### Visualize comapnies

In [17]:
# 'Root' category for all companies venues
companies_category = ['4d4b7105d754a06375d81259', '4d4b7105d754a06378d81259', '4d4b7105d754a06379d81259',
                     '4d4b7104d754a06370d81259']

# Try to load from local file system in case we did this before
meta_company = {}
neighborhoods_company = []
loaded = False
try:
    with open('meta_company_350.pkl', 'rb') as f:
        meta_company = pickle.load(f)
    with open('neighborhoods_company_350.pkl', 'rb') as f:
        neighborhoods_company = pickle.load(f)
    print('Companies data loaded.')
    loaded = True
except:
    pass

if not loaded:
    meta_company, neighborhoods_company = get_meta_venues(latitudes, longitudes, companies_category)
    
    # Let's persists this in local file system
    with open('meta_company_350.pkl', 'wb') as f:
        pickle.dump(meta_company, f)
    with open('neighborhoods_company_350.pkl', 'wb') as f:
        pickle.dump(neighborhoods_company, f)
        
print('Total number of companies:', len(meta_company))
print('Average number of companies in neighborhood:', np.array([len(r) for r in neighborhoods_company]).mean())

map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.Marker(Yialn_center, popup='Prefecture').add_to(map_Yialn)
for res in meta_company.values():
    lat = res[2]; lon = res[3]
    is_company = res[6]
    color = 'red'
    label = '{}'.format(res[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_Yialn)
map_Yialn

Obtaining venues:   0%|          | 0/50 [00:00<?, ? coord/s]

Total number of companies: 167
Average number of companies in neighborhood: 1.22


### Visualize potential competitors and potnital customers. Green are companies, yello is universities, red are competitors.

In [20]:
map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.Marker(Yialn_center, popup='Prefecture').add_to(map_Yialn)
for res in meta_company.values():
    lat = res[2]; lon = res[3]
    color = 'green'
    label = '{}'.format(res[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_Yialn)
for res in meta_univ.values():
    lat = res[2]; lon = res[3]
    color = 'yellow'
    label = '{}'.format(res[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_Yialn)
for res in stores.values():
    lat = res[2]; lon = res[3]
    is_convenience = res[6]
    color = 'red' if is_convenience else 'blue'
    label = '{}'.format(res[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_Yialn)
map_Yialn

### Visualize customer group

In [21]:
# 'Root' category for all companies venues
customer_category = ['4d4b7105d754a06375d81259', '4d4b7105d754a06379d81259',
                     '4d4b7104d754a06370d81259', '4d4b7105d754a06372d81259']

# Try to load from local file system in case we did this before
meta_customers = []
neighborhoods_customers = []
loaded = False
try:
    with open('meta_customers_350.pkl', 'rb') as f:
        meta_customers = pickle.load(f)
    with open('neighborhoods_customers_350.pkl', 'rb') as f:
        neighborhoods_customers = pickle.load(f)
    print('Companies data loaded.')
    loaded = True
except:
    pass

if not loaded:
    from tqdm.autonotebook import tqdm
    pbar1 = tqdm(total=len(customer_category), desc = 'Cycling categories', unit= ' categories')
    for category in customer_category:
        meta_customer, neighborhoods_customer = get_meta_venues(latitudes, longitudes, [category])
        meta_customers.append(meta_customer)
        neighborhoods_customers.append(neighborhoods_customer)
        pbar1.update(1)
    pbar1.close()
    
    # Let's persists this in local file system
    with open('meta_customers_350.pkl', 'wb') as f:
        pickle.dump(meta_customers, f)
    with open('neighborhoods_customers_350.pkl', 'wb') as f:
        pickle.dump(neighborhoods_customers, f)
        
print('Total number of customers:', len(meta_customers))
print('Average number of customers in neighborhood:', np.array([len(r) for r in neighborhoods_customers]).mean())

map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.Marker(Yialn_center, popup='Prefecture').add_to(map_Yialn)
for meta_customer in meta_customers:
    for res in meta_customer.values():
        lat = res[2]; lon = res[3]
        color = 'red'
        label = '{}'.format(res[1])
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                            popup=label, fill_opacity=1, parse_html=False).add_to(map_Yialn)
map_Yialn

Cycling categories:   0%|          | 0/4 [00:00<?, ? categories/s]

Obtaining venues:   0%|          | 0/50 [00:00<?, ? coord/s]

Obtaining venues:   0%|          | 0/50 [00:00<?, ? coord/s]

Obtaining venues:   0%|          | 0/50 [00:00<?, ? coord/s]

Obtaining venues:   0%|          | 0/50 [00:00<?, ? coord/s]

Total number of customers: 4
Average number of customers in neighborhood: 50.0


# Analysis 

### Count the number of business in every area candidate

In [22]:
counts = []
for i, neighborhoods_customer in enumerate(neighborhoods_customers):
    counts.append([len(res) for res in neighborhoods_customers[i]])
final_count= []
len(counts[0])
for m in range(len(counts[0])):
    final_count.append(0)
    for n in range(len(counts)):
        final_count[m] = final_count[m] + counts[n][m]
len(final_count)


50

In [23]:
# location_customers_count = [len(res) for res in neighborhoods_customers]
counts = []
for i, neighborhoods_customer in enumerate(neighborhoods_customers):
    counts.append([len(res) for res in neighborhoods_customers[i]])
location_customers_count= []
len(counts[0])
for m in range(len(counts[0])):
    location_customers_count.append(0)
    for n in range(len(counts)):
        location_customers_count[m] = location_customers_count[m] + counts[n][m]

df_locations['Customers in area'] = location_customers_count

print('Average number of customers in every area with radius=100m:', np.array(location_customers_count).mean())

df_locations.head(5)

Average number of customers in every area with radius=100m: 0.78


,Address,Latitude,Longitude,X,Y,Distance from center,Customers in area
0,"No. 21, Huanhe Road, Yilan City, Yilan County,...",24.764210,121.753484,8.981559e+06,1.353753e+07,1140.175425,0
1,"260, Taiwan, Yilan County, Yilan City, 碧霞街146號",24.762748,121.752353,8.981959e+06,1.353753e+07,860.232527,0
2,"260, Taiwan, Yilan County, Yilan City, 舊城西路125...",24.761287,121.751223,8.982359e+06,1.353753e+07,707.106781,0
3,"No. 109, Jiucheng West Road, Yilan City, Yilan...",24.759826,121.750092,8.982759e+06,1.353753e+07,761.577311,0
4,"No. 142, Wenchang Road, Yilan City, Yilan Coun...",24.758364,121.748961,8.983159e+06,1.353753e+07,989.949494,1


### 10 first areas

In [24]:
df_locations.sort_values(by='Customers in area', ascending=False).head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Customers in area
35,"No. 39, Jiucheng South Road, Yilan City, Yilan...",24.754171,121.752913,8.983459e+06,1.353873e+07,1118.033989,6
11,"No. 276, Section 3, Zhongshan Road, Yilan City...",24.762081,121.754236,8.981859e+06,1.353793e+07,670.820393,5
26,"No. 2, Guorong Road, Yilan City, Yilan County,...",24.753378,121.749900,8.983959e+06,1.353833e+07,1503.329638,2
23,"No. 166, Xinmin Rd, Yilan City, Yilan County, ...",24.757761,121.753292,8.982759e+06,1.353833e+07,316.227766,2
9,"No. 8, Yizhong Road, Yilan City, Yilan County,...",24.751059,121.743310,8.985159e+06,1.353753e+07,2789.265136,2
20,"No. 171號, Jixiang Road, Yilan City, Yilan Coun...",24.762145,121.756685,8.981559e+06,1.353833e+07,905.538514,2
41,"No. 8, Xinxing Road, Yilan City, Yilan County,...",24.758618,121.758755,8.981959e+06,1.353913e+07,1029.563014,2
43,"No. 39, Kangle Road, Yilan City, Yilan County,...",24.755695,121.756492,8.982759e+06,1.353913e+07,948.683298,1
32,"No. 88, Shenghou Street, Yilan City, Yilan Cou...",24.758555,121.756306,8.982259e+06,1.353873e+07,538.516481,1
31,"No. 22-6, Xiaodong Road, Yilan City, Yilan Cou...",24.760016,121.757437,8.981859e+06,1.353873e+07,781.024968,1


In [25]:
store_latlons = [[res[2], res[3]] for res in stores.values()]

customers_latlons = []
for meta_customer in meta_customers:
    customers_latlons.append([[res[2], res[3]] for res in meta_customer.values()])

### The mains areas are in the north of prefecture, clearly identify two big cluster, on north, on east.

In [26]:
from folium import plugins
from folium.plugins import HeatMap

map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(map_Yialn) #cartodbpositron cartodbdark_matter
for customers_latlon in customers_latlons:
    HeatMap(customers_latlon).add_to(map_Yialn)
# folium.Marker(Yialn_center).add_to(map_Yialn)
folium.Circle(Yialn_center, radius=100, fill=False, color='white').add_to(map_Yialn)
folium.Circle(Yialn_center, radius=300, fill=False, color='white').add_to(map_Yialn)
folium.Circle(Yialn_center, radius=500, fill=False, color='white').add_to(map_Yialn)
map_Yialn

### To create another heatmap map showing heatmap/density of convenience stores.

In [27]:
map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(map_Yialn) #cartodbpositron cartodbdark_matter
HeatMap(store_latlons).add_to(map_Yialn)
folium.Marker(Yialn_center, popup='Prefecture').add_to(map_Yialn)
folium.Marker([24.751059, 121.743310], popup='La boite a Bobun',
    icon=folium.Icon(color='red', icon='info-sign')).add_to(map_Yialn)
folium.Circle(Yialn_center, radius=100, fill=False, color='white').add_to(map_Yialn)
folium.Circle(Yialn_center, radius=300, fill=False, color='white').add_to(map_Yialn)
folium.Circle(Yialn_center, radius=500, fill=False, color='white').add_to(map_Yialn)
map_Yialn

### K mean clustering

In [30]:
# We plot the area where we'll search for good localisation
roi_x_min = Yialn_center_x -1000
roi_y_max = Yialn_center_y
roi_width = 1500
roi_height = 1500
roi_center_x = roi_x_min
roi_center_y = roi_y_max
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
for customers_latlon in customers_latlons:
    HeatMap(customers_latlon).add_to(map_Yialn)
folium.Marker(Yialn_center).add_to(map_Yialn)
folium.Circle(Yialn_center, radius=700, color='white', fill=True, fill_opacity=0.4).add_to(map_Yialn)
map_Yialn

In [31]:
k = 2 #math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
nb_k = 20 #51 a la base
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 700

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(nb_k/k)):
    y = roi_y_min + i * y_step
    x_offset = (nb_k-1) if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

246 candidate neighborhood centers generated.


### calculate two most important things for each location candidate: number of competitors in yilan and number of customers.

In [35]:
def count_stores_nearby(x, y, stores, radius=150):    
    count = 0
    for res in stores.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_store(x, y, stores):
    d_min = 100000
    for res in stores.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

def count_customers_nearby(x, y, customers, radius=150):    
    count = 0
    for meta_customer in meta_customers:
        for res in meta_customer.values():
            res_x = res[7]; res_y = res[8]
            d = calc_xy_distance(x, y, res_x, res_y)
            if d<=radius:
                count += 1
    return count

roi_store_counts = []
roi_convenience_stores = []
roi_customer = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_stores_nearby(x, y, stores, radius=100)
    roi_store_counts.append(count)
    
    distance = find_nearest_store(x, y, convenience_stores)
    roi_convenience_stores.append(distance)
    
    custom = count_customers_nearby(x, y, meta_customers)
    roi_customer.append(custom)
print('done.')

Generating data on location candidates... done.


In [36]:
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'stores nearby':roi_store_counts,
                                 'Distance to convenience store':roi_convenience_stores,
                                 'Customers':roi_customer})

df_roi_locations.sort_values(by='Customers', ascending=False).head(10)
# df_roi_locations.head()

,Latitude,Longitude,X,Y,stores nearby,Distance to convenience store,Customers
28,24.762598,121.753436,8.981859e+06,1.353773e+07,0,416.641773,5
54,24.761646,121.753900,8.981978e+06,1.353793e+07,0,221.945936,5
53,24.762012,121.754183,8.981878e+06,1.353793e+07,0,312.449848,5
169,24.754102,121.752860,8.983478e+06,1.353873e+07,0,531.565168,5
170,24.753737,121.752577,8.983578e+06,1.353873e+07,0,478.726010,5
121,24.754404,121.750695,8.983678e+06,1.353833e+07,2,119.938722,5
44,24.756752,121.748913,8.983459e+06,1.353773e+07,0,659.132286,4
235,24.755544,121.757575,8.982659e+06,1.353933e+07,0,220.664094,4
52,24.762377,121.754465,8.981778e+06,1.353793e+07,0,407.418478,4
209,24.756722,121.757287,8.982478e+06,1.353913e+07,0,232.243002,4


### filter those locations: we're interested only in locations with no more than two restaurants in radius of 250 meters, and no convenience stores in radius of 50 meters, and more than 3 customers.

In [50]:
good_res_count = np.array((df_roi_locations['stores nearby']<=2))
print('Locations with no more than two stores nearby:', good_res_count.sum())

good_asi_distance = np.array(df_roi_locations['Distance to convenience store']>=50)
print('Locations with no convenience stores within 100m:', good_asi_distance.sum())

good_custmer_count = np.array(df_roi_locations['Customers']>=3)
print('Locations with more than 10 customers:', good_custmer_count.sum())

good_locations = np.logical_and(good_custmer_count, good_res_count, good_asi_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]

Locations with no more than two stores nearby: 246
Locations with no convenience stores within 400m: 243
Locations with more than 10 customers: 28
Locations with both conditions met: 28


### Visualize result

In [51]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(map_Yialn)
HeatMap(store_latlons).add_to(map_Yialn)
folium.Circle(Yialn_center, radius=700, color='white', fill=True, fill_opacity=0.6).add_to(map_Yialn)
folium.Marker(Yialn_center).add_to(map_Yialn)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, 
                        fill_color='blue', fill_opacity=1).add_to(map_Yialn) 
map_Yialn

### identify three mains areas, one is south, on is east and the other one north.

In [53]:
map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
HeatMap(good_locations, radius=35).add_to(map_Yialn)
folium.Marker(Yialn_center).add_to(map_Yialn)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, 
                        fill_color='blue', fill_opacity=1).add_to(map_Yialn)
map_Yialn

### cluster those locations to create centers of zones containing good locations

In [54]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_Yialn = folium.Map(location=Yialn_center, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(map_Yialn)
for customers_latlon in customers_latlons:
    HeatMap(customers_latlon).add_to(map_Yialn)
folium.Circle(Yialn_center, radius=700, color='white', fill=True, fill_opacity=0.4).add_to(map_Yialn)
folium.Marker(Yialn_center).add_to(map_Yialn)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=80, color='green', fill=True, fill_opacity=0.25).add_to(map_Yialn) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', 
                        fill_opacity=1).add_to(map_Yialn)
map_Yialn

### see those zones on a city map without heatmap, using shaded areas to indicate our clusters

In [57]:
map_Yialn = folium.Map(location=[24.751059, 121.743310], zoom_start=16)
folium.Marker(Yialn_center).add_to(map_Yialn)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=60, color='green', fill=False).add_to(map_Yialn) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', 
                  fill_opacity=0.07).add_to(map_Yialn)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', 
                        fill_opacity=1).add_to(map_Yialn)
map_Yialn

### Finally, reverse geocode those candidate area centers to get the addresses

In [58]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(GOOGLE_API_KEY, lat, lon).replace(', France', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, Yialn_center_x, Yialn_center_y)
    print('{}{} => {:.1f}km from Prefecture'.format(addr, ' '*(50-len(addr)), d/1000))

Addresses of centers of areas recommended for further analysis

No. 38巷2號, Section 2, Minquan Road, Yilan City, Yilan County, Taiwan 260 => 1.3km from Prefecture
No. 19, Shenghou Street, Yilan City, Yilan County, Taiwan 260 => 1.0km from Prefecture
No. 116, Xihou Street, Yilan City, Yilan County, Taiwan 260 => 0.3km from Prefecture
No. 274號, Section 3, Zhongshan Road, Yilan City, Yilan County, Taiwan 260 => 0.7km from Prefecture
No. 45, Hemu Road, Yilan City, Yilan County, Taiwan 260 => 0.9km from Prefecture
No. 3, Jiucheng West Road, Yilan City, Yilan County, Taiwan 260 => 1.1km from Prefecture
No. 516號, Section 2, Zhongshan Road, Yilan City, Yilan County, Taiwan 260 => 1.2km from Prefecture
No. 366, Section 1, Yixing Road, Yilan City, Yilan County, Taiwan 260 => 1.2km from Prefecture
No. 255, Section 1, Yixing Road, Yilan City, Yilan County, Taiwan 260 => 1.1km from Prefecture
No. 69, Tongxing St, Yilan City, Yilan County, Taiwan 260 => 0.7km from Prefecture
No. 160, Section 3, Zhong